The goal of this notebook is to apply a model of dormancy to the model used in Constable and McKane 2015.

In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

The model used here is similar in many ways, however, some clarification is necessary to understand the math below. We are interested in the dynamics of a dormant subpopulation and an active population. We start with a population of $n_{A}$ active individuals and $n_{D}$ dormant individuals.  The active subpopulation reproduces at a rate of $b_{A}$ and both subpopulations reproduce at rates of $d_{A}$ and $d_{D}$. Dormant individuals do not have a rate of reproduction as they cannot reproduce. We allow for compeition at rates of either $c_{AD}$ or $c_{DA}$, depending on the equation.

We are going with the rules laid out in Constable and McKane of setting up  a well-mixed system with the state specified by $\mathbf{n} = (n_{A}, n_{D})$. Below we define the rates at which active and dormant subpopulations reproduce, die, switch states by resusitation, or engage in competition. 

$(1)$

$$T_{1}(n_{A}+1, n_{D}\mid n_{A}, n_{D}) = b_{A}\frac{n_{A}}{V}$$
$$T_{2}(n_{A}, n_{D}+1\mid n_{A}, n_{D}) = b_{D}\frac{n_{D}}{V} = 0$$
$$T_{3}(n_{A}-1, n_{D}\mid n_{A}, n_{D}) = d_{A}\frac{n_{A}}{V} + c_{AA}\frac{n_{A}}{V}\frac{n_{A}}{V}+c_{AD}\frac{n_{D}}{V}\frac{n_{A}}{V}$$
$$T_{4}(n_{A}, n_{D}-1\mid n_{A}, n_{D}) = d_{D}\frac{n_{D}}{V} + c_{DD}\frac{n_{D}}{V}\frac{n_{D}}{V}+c_{DA}\frac{n_{A}}{V}\frac{n_{D}}{V}$$
$$T_{5}(n_{A}+1, n_{D}-1\mid n_{A}, n_{D}) = R\frac{n_D}{V}$$
$$T_{6}(n_{A}-1, n_{D}+1\mid n_{A}, n_{D}) = (1-R)\frac{n_A}{V}$$ 

The parameter $V$ is a measure of the size of the system. Using the logic form Constable and McKane 2015, this parameter will be absorved into the rates $b_{i}$, $d_{i}$, and $c_{ij}$. The probability of finding the system in state $\mathbf{n}$ at time $t$, $P_{n}(t)$ can be found using the master equaiton: 

$(2)$

$$\frac{dP_{n}(t)}{dt} = \sum_{\mu =1}^{6}[T_{\mu}(n\mid n-v_{\mu})P_{n-v_{\mu}}(t)-T_{\mu}(n+v_{\mu}\mid n)P_{n}(t)]$$

where $v_{\mu}$ represents the number of individuals of one type that are transformed during the reaction $\mu=1,...,6$, so 
$v_{1} = (1,0)$,
$v_{2} = (0,0)$,
$v_{3} = (-1,0)$,
$v_{4} = (0,-1)$,
$v_{5} = (1,-1)$,
$v_{6} = (-1,1)$.

We can find $P_{n}(t)$ for all t using the two equations above and an initial condition for $P_{n}.$ However, the master equaiton is intractable, so a diffusion approximation is done by first assuming that V is large enough that $x_{i} \equiv n_{i}/V$ is continuous. Next we expand the master equation as a power series in $V^{-1}$ to get the Fokker-Planck equation: 

$(3)$

$$\frac{\partial P(x,\tau )}{\partial \tau} = -\sum_{i =1}^{2}\frac{\partial }{\partial x_{i}}[A_{i}(x)P(x, \tau)] + \frac{1}{2V}\sum_{i,j=1}^{2}\frac{\partial^2 }{\partial x_{i} \partial x_{j}}[B_{ij}(x)P(x, \tau)],$$

Where $\tau = t/V$ is rescaled time and higher-order terms have been rejected. 
Functions $A_{i}$ and $B_{ij}$ can be expressed in terms of the $v_{i, \mu}$ and function $f_{u}$ as:

$(4)$

$$A_{i}(x)= \sum_{\mu=1}^{6}v_{i, \mu}f_{\mu}(x), \; B_{ij}= \sum_{\mu=1}^{6}v_{i,\mu}v_{j,\mu}f_{\mu}(x),$$

Where $$i,j = 1,2$$ and where functions $f_{\mu}(x)$ are equal to $T_{\mu}(Vx+v_{\mu} \mid Vx)$. So, since we are dealing with either active or dormant subpopulations, we get the following from the above two equations:

$$A_{A}(x)=\sum_{\mu=1}^{6}v_{A, \mu}f_{\mu}(x), \; A_{D}(x)=\sum_{\mu=1}^{6}v_{D, \mu}f_{\mu}(x), \; B_{AD}= \sum_{\mu=1}^{6}v_{A,\mu}v_{D,\mu}f_{\mu}(x),$$

Unlike Kimura's (and others) use of FPE's for diffusion approximation,we are using the equivalent Ito SDE. 

$(5)$

$$\frac{\mathrm{d}x_{i} }{\mathrm{d} \tau} = A_{i}(x)+\frac{1}{\sqrt{V}}\eta _{i} (\tau),$$

where $\eta _{i} (\tau)$ is Gaussian noise with 

$(6)$

$$\left \langle \eta _{i} (\tau) \right \rangle = 0, \; \left \langle \eta _{i} (\tau)\eta _{j} (\tau') \right \rangle = B_{ij}(x)\delta (\tau-\tau')$$

We can get the form of $A_{i}(x)$ and $B_{ij}(x)$ using Eq. (1), Eq. (4), and $v_{i, \mu}$. From that, we get:

$(7)$

$$A_{A}(x)=(b_{A}-d_{A})x_{A}-c_{AA}x_{A}^{2}-c_{AD}x_{A}x_{D}+Rx_{D}-(1-R)x_{A}$$
$$A_{D}(x)=-d_{D}x_{D}-c_{DD}x_{D}^{2}-c_{DA}x_{D}x_{A}-Rx_{D}+(1-R)x_{A}$$
$$B_{AA}(x)=(b_{A}+d_{A})x_{A}+c_{AA}x_{A}^{2}+c_{AD}x_{A}x_{D}+Rx_{D}+(1-R)x_{A}$$
$$B_{DD}(x)=d_{D}x_{D}+c_{DD}x_{D}^{2}+c_{DA}x_{D}x_{A}+Rx_{D}+(1-R)x_{A}$$

Where $B_{ij}=0$ for all $i\neq j$. As $V \rightarrow \infty$, Eq. (5) reduces to the two deterministic differential equations $\frac{\mathrm{d} x_{i}}{\mathrm{d} \tau} = A_{i}(x)$ with $A_{i}$ given by Eq. (7). These are the **Lotka-Volterra** equations for two competiting species (subpopulations in this case).

To start, we assume that A_{A} and A_{D} have equal fitness. Say they have equal death and competition rates and assign the active subpopulation some birth rate. 

We want to look for fixed points of the dynamics by setting $A_{i}(x) = 0, i = A,D$. By taking the combinations $A_{A}\pm A_{D}$, we find that the fixed points are solutions of the two equations. 

$(8)$

$$[x_{A}b_{A}-(d_{0}+c_{0}(x_{A}+x_{D}))(x_{A}\pm x_{D})] = 0$$

We can see that because the biological underpinnings of the model rest on dormant individuals not reproducing, that  $b_{A}$ should play an important role in the model. By rearranging Eq. (8), we get:

$(9)$

$$x_{A} + x_{D} = c_{0}^{-1} \left ( b_{A} \left ( 1 + \frac{x_{A}}{x_{D}} \right )  -d_{0} \right )$$

What to do next? Skip to non-neutral systems or.....

In [8]:
# Set limits and number of points in grid
xD, xA = np.mgrid[2.5:0:100j, 2.5:0:100j]


In [7]:
b_A

2